# Experiment 05-08

In [14]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging
from pqdm.threads import pqdm
from tqdm.notebook import tqdm

## Configure Logging

In [15]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_08.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-13 18:22:19 INFO     Started


## Show the available Platforms

In [16]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: AMD Accelerated Parallel Processing
Platform 0 - Vendor: Advanced Micro Devices, Inc.
Platform 0 - Version: OpenCL 2.1 AMD-APP (3354.13)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1675 Mhz
 Device - Compute Units: 18
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256


 --------------------------------------------------------
 Device - Name: gfx1010:xnack-
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1815 Mhz
 Device - Compute Units: 20
 Device - Local Memory: 64 KB
 Device - Constant Memory: 6893568 KB
 Device - Global Memory: 8 GB
 Device - Max Buffer/Image Size: 6732 MB
 Device - Max Work Group Size: 256




## Configure the OpenCL Context

In [17]:
platform_number = 0
device_number = 1

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

In [18]:
cl_device = cl_devices[device_number]

cl_threads = cl_device.max_compute_units * cl_device.max_work_group_size
if cl_device.type & 4 == 0:
    cl_threads = cl_devices.max_work_group_size

print('Max Compute Units:', cl_device.max_compute_units)
print('Max Compute Units:', cl_device.max_work_group_size)
print('Threads:', cl_threads)

Max Compute Units: 20
Max Compute Units: 256
Threads: 5120


## Compile the Program

In [19]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [20]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: get_random_numbers;mine_eight;get_random_string;get_single_hash_nonce;get_single_hash;hash_main;mine_eight_sequential;mine_nonce


## Mine Nonce

In [21]:
max_nonce = 20
max_output_size = 256

In [22]:
a = '0000000093e0dbb2e58e64502569a8e47c46da1e15608bcb78812ab6376b9634'

In [23]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray, plaintext : str, prefix : str) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, max_output_size):
        if nonce_len[i] > 0:
            nonce_str = prefix + nonce[i * max_nonce:i * max_nonce + nonce_len[i]].tobytes().decode('UTF-8')
            zeros_found[i] = nonce_str
            hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))
            zeros = len(hash.hexdigest()) - len(hash.hexdigest().lstrip('0'))

            logging.info("%4d: [%2d] %20s %64s" % (i, zeros, nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [24]:
print(np.iinfo(np.unsignedinteger).max)
print(np.iinfo(np.ulonglong).max)
print(len(str(np.iinfo(np.ulonglong).max)))

4294967295
18446744073709551615
20


In [25]:
#cl_threads = (cl_threads // 2)
#cl_threads = 1

In [26]:
postfix = ""
base_text = 'this is a description of the latest block'
plaintext = base_text + postfix
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)

# set up the variables to generate the random numbers
seed = np.ulonglong(1)
window_size = np.uint32(1000000)
nonce = np.zeros(shape=max_nonce * max_output_size, dtype=np.uint8)
nonce_len = np.zeros(shape=max_output_size, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_eight_sequential(
        cl_queue, (cl_threads,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len
    )

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # display the last nonce tried for each thread
    # for i in range(0, cl_threads):
    #     logging.info("Thread %d: %d" % (i, last_nonce[i]))

    # for i in range(0, nonce.size):
    #     print(nonce[i], end=' ')

    # display the results
    display_output(zeros_found, nonce, nonce_len, base_text, postfix)
    logging.info('Iteration Complete.')

    seed = np.ulonglong(seed + cl_threads * window_size)
    keep_running = True

2022-07-13 18:22:37 INFO        0: [ 8]            395707976 0000000053d18bbf3e8293e6a3bacda6e35ee7715e6353bf55a86cb9c3b4167b
2022-07-13 18:22:37 INFO     Iteration Complete.


In [ ]:
input_buffer = {}
nonce = {}
nonce_len = 0
seed = 123456

while (seed > 0):
    nonce_len += 1
    input_buffer[nonce_len - 1] = (seed % 10)
    seed = seed // 10

print(input_buffer)

for i in range(0, nonce_len):
    nonce[nonce_len-1 - i] = input_buffer[i]

for i in range(0, nonce_len):
    print(nonce[i])
    

{0: 6, 1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
1
2
3
4
5
6
